In [3]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from torch.optim import AdamW
from transformers import RobertaForSequenceClassification
from sklearn.metrics import f1_score
import numpy as np
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

import torch
from torch.optim import AdamW
from transformers import RobertaForSequenceClassification
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path 
import os 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
import torch

In [2]:
#Se leen los datos y se seleccionan las variables que presentan una diferencia
# entre los individuos que presentan estres y los que no
data_path = Path(os.getcwd()).parent / "data"
silver_path = data_path / "silver" / "dreadditCleanTrain.csv"
df = pd.read_csv(silver_path, usecols= ["text","clean_text","clean_text_sentence_sep",
                                        "singular_pronouns", "avg_word_len",'lex_diversity',"label"])

In [4]:

model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

c:\Users\franc\miniconda3\envs\tesis\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\franc\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment-latest. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentime

In [5]:
# Ejemplo de cómo usar el dataset
dataset = SentimentDataset(df['clean_text'].tolist(), df['label'].tolist(), tokenizer, max_length=128)
loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [6]:
# Suponiendo que 'model' y 'loader' están definidos y configurados correctamente
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

# Listas para almacenar las métricas
losses = []
f1_scores = []

In [7]:

def train_and_evaluate(model, train_loader, val_loader, device, num_epochs=2):
    optimizer = AdamW(model.parameters(), lr=1e-5)
    best_f1 = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            train_loss += loss.item()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_loader)

        # Evaluación
        model.eval()
        val_true, val_preds = [], []
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)

                val_true.extend(labels.cpu().numpy())
                val_preds.extend(predictions.cpu().numpy())

        val_f1 = f1_score(val_true, val_preds, average='binary')
        if val_f1 > best_f1:
            best_f1 = val_f1  # Actualizar el mejor F1 score visto hasta ahora

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation F1: {val_f1:.4f}')

    return best_f1


In [8]:
from torch.utils.data import Subset, DataLoader

# Inicializar el KFold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Convertir labels a numpy para el uso con StratifiedKFold
labels = np.array(df['label'].tolist())

# Ejecutar la validación cruzada
f1_scores = []
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    train_subsampler = Subset(dataset, train_idx)
    val_subsampler = Subset(dataset, val_idx)
    
    train_loader = DataLoader(train_subsampler, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_subsampler, batch_size=16, shuffle=False)

    print(f"Training fold {fold+1}")
    fold_f1 = train_and_evaluate(model, train_loader, val_loader, device)
    f1_scores.append(fold_f1)

print(f"Average F1 across folds: {np.mean(f1_scores):.4f}")

Training fold 1
Epoch 1/2, Train Loss: 0.6189, Validation F1: 0.8442


KeyboardInterrupt: 

In [ ]:
def save_model_and_tokenizer(model, tokenizer, save_directory):
    """
    Save the fine-tuned model and tokenizer to a specified directory.

    Args:
        model: The trained RoBERTa model (e.g., an instance of RobertaForSequenceClassification).
        tokenizer: The tokenizer used with the RoBERTa model.
        save_directory (str): The path to the directory where the model and tokenizer will be saved.

    Returns:
        None
    """
    # Create the directory if it does not exist
    import os
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # Save the model
    model.save_pretrained(save_directory)
    
    # Save the tokenizer associated with the model
    tokenizer.save_pretrained(save_directory)

    print(f"Model and tokenizer have been saved to {save_directory}") 